In [2]:
!pip install geocoder

     |████████████████████████████████| 98 kB 5.3 MB/s 


In [3]:
import json
import geocoder
import math
import csv
import numpy as np
import datetime
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/"MyDrive"/"Colab Notebooks"/"DataMining"
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/DataMining
 manifest.csv				     output_data
'NearBy estate  Data pre-processing.ipynb'   pre_processing_data
'NearBy estate.ipynb'			     房地產資料


In [5]:
csv_filename=[]
with open('manifest.csv') as manifest:
  rows = csv.reader(manifest, delimiter=',')
  headers = next(rows)
  for row in rows:
    if(row[1]=="schema-main.csv"):
      csv_filename.append(row[0])
print(csv_filename)

['a_lvr_land_a.csv', 'b_lvr_land_a.csv', 'c_lvr_land_a.csv', 'd_lvr_land_a.csv', 'e_lvr_land_a.csv', 'f_lvr_land_a.csv', 'g_lvr_land_a.csv', 'h_lvr_land_a.csv', 'i_lvr_land_a.csv', 'j_lvr_land_a.csv', 'k_lvr_land_a.csv', 'm_lvr_land_a.csv', 'n_lvr_land_a.csv', 'o_lvr_land_a.csv', 'p_lvr_land_a.csv', 'q_lvr_land_a.csv', 't_lvr_land_a.csv', 'u_lvr_land_a.csv', 'v_lvr_land_a.csv', 'w_lvr_land_a.csv', 'x_lvr_land_a.csv']


In [6]:
def get_geographic_coordinates(address):
    try:
      ret = geocoder.arcgis(address)
      return(ret.latlng)
    except:
      #print ERROR ADDRESS if address can't be parsed  
      print("ERROR")

In [7]:
def count_age(text):
  try:
    today=datetime.datetime.today()
    if(len(text)==6):
      finish_year=int(text[:2])+1911
      finish_month=text[2:4]
      finish_date=text[4:]
    else:
      finish_year=int(text[:3])+1911
      finish_month=text[3:5]
      finish_date=text[5:]
    finish=datetime.datetime.strptime(f"{finish_year}-{finish_month}-{finish_date}", "%Y-%m-%d")
    return (today-finish).days
  except:
    return 0

In [ ]:
def Postal_code(city,district):
  data={
      '臺北市': {'中正': '100', '大同': '103', '中山': '104', '松山': '105', '大安': '106', '萬華': '108', '信義': '110', '士林': '111', '北投': '112', '內湖': '114', '南港': '115', '文山': '116'}, 
      '基隆市': {'仁愛': '200', '信義': '201', '中正': '202', '中山': '203', '安樂': '204', '暖暖': '205', '七堵': '206'}, 
      '新北市': {'萬里': '207', '金山': '208', '板橋': '220', '汐止': '221', '深坑': '222', '石碇': '223', '瑞芳': '224', '平溪': '226', '貢寮': '227', '新店': '231', '坪林': '232', '烏來': '233', '永和': '234', '中和': '235', '土城': '236', '三峽': '237', '樹林': '238', '鶯歌': '239', '三重': '241', '新莊': '242', '泰山': '243', '林口': '244', '蘆洲': '247', '五股': '248', '八里': '249', '淡水': '251', '三芝': '252', '石門': '253'}, 
      '宜蘭縣': {'宜蘭': '260', '頭城': '261', '礁溪': '262', '壯圍': '263', '員山': '264', '羅東': '265', '三星': '266', '大同': '267', '五結': '268', '冬山': '269', '蘇澳': '270', '南澳': '272'}, 
      '連江縣': {'南竿': '209', '北竿': '210', '莒光': '211', '東引': '212'}, 
      '新竹市': {'香山': '300'}, 
      '新竹縣': {'竹北': '302', '湖口': '303', '新豐': '304', '新埔': '305', '關西': '306', '芎林': '307', '寶山': '308', '竹東': '310', '五峰': '311', '橫山': '312', '尖石': '313', '北埔': '314', '峨眉': '315'}, 
      '桃園市': {'中壢': '320', '平鎮': '324', '龍潭': '325', '楊梅': '326', '新屋': '327', '觀音': '328', '桃園': '330', '龜山': '333', '八德': '334', '大溪': '335', '復興': '336', '大園': '337', '蘆竹': '338'}, 
      '苗栗縣': {'竹南': '350', '頭份': '351', '三灣': '352', '南庄': '353', '獅潭': '354', '後龍': '356', '通霄': '357', '苑裡': '358', '苗栗': '360', '造橋': '361', '頭屋': '362', '公館': '363', '大湖': '364', '泰安': '365', '銅鑼': '366', '三義': '367', '西湖': '368', '卓蘭': '369'}, 
      '臺中市': {'中區': '400', '東區': '401', '南區': '402', '西區': '403', '北區': '404', '北屯': '406', '西屯': '407', '南屯': '408', '太平': '411', '大里': '412', '霧峰': '413', '烏日': '414', '豐原': '420', '后里': '421', '石岡': '422', '東勢': '423', '和平': '424', '新社': '426', '潭子': '427', '大雅': '428', '神岡': '429', '大肚': '432', '沙鹿': '433', '龍井': '434', '梧棲': '435', '清水': '436', '大甲': '437', '外埔': '438', '大安': '439'}, 
      '彰化縣': {'彰化': '500', '芬園': '502', '花壇': '503', '秀水': '504', '鹿港': '505', '福興': '506', '線西': '507', '和美': '508', '伸港': '509', '員林': '510', '社頭': '511', '永靖': '512', '埔心': '513', '溪湖': '514', '大村': '515', '埔鹽': '516', '田中': '520', '北斗': '521', '田尾': '522', '埤頭': '523', '溪州': '524', '竹塘': '525', '二林': '526', '大城': '527', '芳苑': '528', '二水': '529'}, 
      '南投縣': {'南投': '540', '中寮': '541', '草屯': '542', '國姓': '544', '埔里': '545', '仁愛': '546', '名間': '551', '集集': '552', '水里': '553', '魚池': '555', '信義': '556', '竹山': '557', '鹿谷': '558'}, 
      '嘉義市': {'西區': '600'}, 
      '嘉義縣': {'番路': '602', '梅山': '603', '竹崎': '604', '阿里山': '605', '中埔': '606', '大埔': '607', '水上': '608', '鹿草': '611', '太保': '612', '朴子': '613', '東石': '614', '六腳': '615', '新港': '616', '民雄': '621', '大林': '622', '溪口': '623', '義竹': '624', '布袋': '625'}, 
      '雲林縣': {'斗南': '630', '大埤': '631', '虎尾': '632', '土庫': '633', '褒忠': '634', '東勢': '635', '臺西': '636', '崙背': '637', '麥寮': '638', '斗六': '640', '林內': '643', '古坑': '646', '莿桐': '647', '西螺': '648', '二崙': '649', '北港': '651', '水林': '652', '四湖': '653', '元長': '654'}, 
      '臺南市': {'中西區': '700', '東區': '701', '南區': '702', '北區': '704', '安平': '708', '安南': '709', '永康': '710', '歸仁': '711', '新化': '712', '左鎮': '713', '玉井': '714', '楠西': '715', '南化': '716', '仁德': '717', '關廟': '718', '龍崎': '719', '官田': '720', '麻豆': '721', '佳里': '722', '西港': '723', '七股': '724', '將軍': '725', '學甲': '726', '北門': '727', '新營': '730', '後壁': '731', '白河': '732', '東山': '733', '六甲': '734', '下營': '735', '柳營': '736', '鹽水': '737', '善化': '741', '大內': '742', '山上': '743', '新市': '744', '安定': '745'}, 
      '高雄市': {'新興': '800', '前金': '801', '苓雅': '802', '鹽埕': '803', '鼔山': '804', '旗津': '805', '前鎮': '806', '三民': '807', '楠梓': '811', '小港': '812', '左營': '813', '仁武': '814', '大社': '815', '岡山': '820', '路竹': '821', '阿蓮': '822', '田寮': '823', '燕巢': '824', '橋頭': '825', '梓官': '826', '彌陀': '827', '永安': '828', '湖內': '829', '鳳山': '830', '大寮': '831', '林園': '832', '鳥松': '833', '大樹': '840', '旗山': '842', '美濃': '843', '六龜': '844', '內門': '845', '杉林': '846', '甲仙': '847', '桃源': '848', '那瑪夏': '849', '茂林': '851', '茄定': '852'}, 
      '澎湖縣': {'馬公': '880', '西嶼': '881', '望安': '882', '七美': '883', '白沙': '884', '湖西': '885'}, 
      '金門縣': {'金沙': '890', '金湖': '891', '金寜': '892', '金城': '893', '烈嶼': '894', '烏坵': '896'}, 
      '屏東縣': {'竹田': '911', '三地門': '901', '霧臺': '902', '瑪家': '903', '九如': '904', '里港': '905', '高樹': '906', '鹽埔': '907', '長治': '908', '麟洛': '909', '內埔': '912', '萬丹': '913', '潮州': '920', '泰武': '921', '來義': '922', '萬巒': '923', '崁頂': '924', '新埤': '925', '南州': '926', '林邊': '927', '東港': '928', '珫球': '929', '佳冬': '931', '新園': '932', '枋寮': '940', '枋山': '941', '春日': '942', '獅子': '943', '車城': '944', '牡丹': '945', '恒春': '946', '滿州': '947'}, 
      '臺東縣': {'臺東': '950', '綠島': '951', '蘭嶼': '952', '延平': '953', '卑南': '954', '鹿野': '955', '關山': '956', '海端': '957', '池上': '958', '東河': '959', '成功': '961', '長濱': '962', '太麻里': '963', '金峰': '964', '大武': '965', '達仁': '966'}, 
      '花蓮縣': {'花蓮': '970', '新城': '971', '秀林': '972', '吉安': '973', '壽豐': '974', '鳳林': '975', '光復': '976', '豐演': '977', '瑞穗': '978', '萬榮': '979', '玉里': '981', '卓溪': '982', '富里': '983'}
    }
  return data[city][district]


In [ ]:
category={
    "transaction_sign":{
      "num":0,
    },
    "type":{
      "num":0,
    },
    "building_state":{
      "num":0,
    },
    "main_use":{
      "num":0,
    },
    "building_materials":{
      "num":0,
    },
    "berth_category":{
      "num":0,
    },
  }

In [ ]:
def trans(title,text):
  if not (text in category[title]):
    num = category[title]["num"]
    category[title][text]= num+1
    category[title]["num"]=num+1
    return category[title][text]
  else:
    return category[title][text]

In [ ]:
def chinese_floor(text):
  if text=="全":
    return 0
  else:
    digit = {'一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十': 10}
    # 位數遞增，由高位開始取
    digit_num = 0
    # 結果
    result = 0
    # 暫時存儲的變量
    tmp = 0
    while digit_num < len(text):
      tmp_zh = text[digit_num]
      tmp_num = digit.get(tmp_zh, None)
      if tmp_num == 10:
        if tmp == 0:
          tmp = 1
        result = result + tmp_num * tmp
        tmp = 0
      elif tmp_num is not None:
        tmp = tmp * 10 + tmp_num
      digit_num += 1
    result = result + tmp
    return result


將符合的資料整理起來

In [ ]:
tot_num_data=0
data={}
csv_tot_data=[]
#for i in tqdm(csv_filename):
num_data=0
csv_data=[]
%cd /content/drive/"MyDrive"/"Colab Notebooks"/"DataMining"/"房地產資料"
with open("b_lvr_land_a.csv", newline='') as estate:
  rows = csv.reader(estate, delimiter=',')
  headers = next(rows)
  headers = next(rows)
  for row in rows:
    if("房地(土地+建物)" in row[1]):
      latlng_data=get_geographic_coordinates(row[2])
      if(latlng_data):
        #print(row)
        num_data+=1
        tot_num_data+=1
        data={
            "data_num":num_data,
            "postal_code":Postal_code(row[2][:3],row[0][:-1] if len(row[0])>2 else row[0]),
            "transaction_sign":trans("transaction_sign",row[1]),
            #"address":row[2],
            "land_shifting_total_area":row[3],
            "building_shifting_total_area":row[15],
            "type":trans("type",row[4]),
            "transaction_land_num":row[8][2],
            "transaction_building_num":row[8][5],
            "transaction_berth_num":row[8][8],
            "shifting_level":row[9],
            "total_floor":chinese_floor(row[10][:-1]),
            "building_state":trans("building_state",row[11]),
            "main_use":trans("main_use",row[12]),
            "building_materials":trans("building_materials",row[13]),
            "building_age":count_age(row[14]),
            "building_room_num":row[16],
            "building_hall_num":row[17],
            "building_bathroom_num":row[18],
            "building_compartmented_num": 1 if row[19]=="有" else 0,
            "residential_guard":1 if row[20]=="有" else 0,
            "building_price":row[21],
            "unit_price":row[22],
            "berth_category":trans("berth_category",row[23]) if row[23] else 0,
            "berth_area":row[24],
            "berth_price":row[25],
            "note":row[26],
            "elevator":1 if row[31]=="有" else 0,
            "lat":latlng_data[0],
            "lng":latlng_data[1],
        }
        #print(data)
        csv_data.append(data)
#           data["data_num"]=tot_num_data
#           csv_tot_data.append(data)
  #print(csv_data)
  print(len(csv_data))
  %cd /content/drive/"MyDrive"/"Colab Notebooks"/"DataMining"/"pre_processing_data"
  with open('b_lvr_land_a.csv.json', 'w') as f:
    json.dump(csv_data, f)
# #print(csv_tot_data)
# print(len(csv_tot_data))
# with open("output.json", 'w') as f:
#   json.dump(csv_tot_data, f)

/content/drive/MyDrive/Colab Notebooks/DataMining/房地產資料
907
/content/drive/MyDrive/Colab Notebooks/DataMining/pre_processing_data


In [ ]:
%cd /content/drive/"MyDrive"/"Colab Notebooks"/"DataMining"/"pre_processing_data"
with open("b_lvr_land_a.csv.json", newline='') as f:
   data = json.load(f)
   for estate in tqdm(data):
     print(estate)

/content/drive/MyDrive/Colab Notebooks/DataMining/pre_processing_data


100%|██████████| 907/907 [00:00<00:00, 8238.78it/s]

{'data_num': 1, 'postal_code': '402', 'transaction_sign': 1, 'land_shifting_total_area': '5.14', 'building_shifting_total_area': '66.84', 'type': 1, 'transaction_land_num': '1', 'transaction_building_num': '1', 'transaction_berth_num': '0', 'shifting_level': '二層', 'total_floor': 22, 'building_state': 1, 'main_use': 1, 'building_materials': 1, 'building_age': 5020, 'building_room_num': '2', 'building_hall_num': '2', 'building_bathroom_num': '2', 'building_compartmented_num': 1, 'residential_guard': 1, 'building_price': '5000000', 'unit_price': '74806', 'berth_category': 0, 'berth_area': '0.0', 'berth_price': '0', 'note': '陽台外推；', 'elevator': 1, 'lat': 24.118790013770948, 'lng': 120.66986999505662}
{'data_num': 2, 'postal_code': '403', 'transaction_sign': 2, 'land_shifting_total_area': '47.58', 'building_shifting_total_area': '486.53', 'type': 1, 'transaction_land_num': '1', 'transaction_building_num': '2', 'transaction_berth_num': '3', 'shifting_level': '八層', 'total_floor': 14, 'buildin

In [ ]:
print(category)
with open('category.json', 'w') as f:
    json.dump(category, f)

{'transaction_sign': {'num': 2, '房地(土地+建物)': 1, '房地(土地+建物)+車位': 2}, 'type': {'num': 34, '都市：其他:第三種住宅區。': 1, '都市：其他:第三種商業區': 2, '都市：其他:第二種住宅區': 3, '都市：其他:第一種商業區': 4, '都市：其他:第四種商業區': 5, '都市：其他:第二種商業區': 6, '非都市： ; 非都市編定：': 7, '都市：其他:第一之B種住宅區': 8, '都市：其他:第三之二種住宅區': 9, '都市：其他:第五種住宅區': 10, '都市：其他:第六種新市政中心專用區': 11, '都市：其他:第六種住宅區': 12, '都市：其他:第一種之一住宅區': 13, '都市：其他:第二之二種住宅區': 14, '都市：其他:第一之C種住宅區': 15, '都市：其他:住宅區': 16, '': 17, '都市：其他:第四種住宅區': 18, '都市：其他:道路用地': 19, '都市：其他:商業區': 20, '都市：其他:乙種工業區': 21, '都市：其他:第一種住宅區': 22, '都市：其他:市場用地': 23, '都市：其他:農業區': 24, '都市：其他:保護區': 25, '都市：其他:第三之一種住宅區': 26, '都市：其他:第三種新市政中心專用區': 27, '都市：其他:第二之一種商業區': 28, '都市：其他:第一之一種住宅區': 29, '都市：其他:甲種工業區': 30, '都市：其他:特一種住宅區': 31, '都市：其他:風景區': 32, '都市：其他:第四之一種商業區': 33, '都市：其他:第二之A種住宅區': 34}, 'building_state': {'num': 4, '住宅大樓(11層含以上有電梯)': 1, '公寓(5樓含以下無電梯)': 2, '華廈(10層含以下有電梯)': 3, '透天厝': 4}, 'main_use': {'num': 88, '集合住宅': 1, '住家用': 2, '機械室，住宅': 3, '住商用': 4, '店舖、住宅、梯間': 5, '店舖、住宅、樓梯間、停車空間、人行道、機電設備空間': 6, '商業用': 7, '店舖、住房、人行道、儲藏室'

In [ ]:
%cd /content/drive/"MyDrive"/"Colab Notebooks"/"DataMining"/"pre_processing_data"
!ls

/content/drive/MyDrive/Colab Notebooks/DataMining/pre_processing_data
a_lvr_land_a.csv.json  i_lvr_land_a.csv.json  q_lvr_land_a.csv.json
b_lvr_land_a.csv.json  j_lvr_land_a.csv.json  t_lvr_land_a.csv.json
c_lvr_land_a.csv.json  k_lvr_land_a.csv.json  too_high1.json
d_lvr_land_a.csv.json  m_lvr_land_a.csv.json  too_low1.json
e_lvr_land_a.csv.json  n_lvr_land_a.csv.json  u_lvr_land_a.csv.json
f_lvr_land_a.csv.json  o_lvr_land_a.csv.json  v_lvr_land_a.csv.json
g_lvr_land_a.csv.json  output.json	      w_lvr_land_a.csv.json
h_lvr_land_a.csv.json  p_lvr_land_a.csv.json  x_lvr_land_a.csv.json
